In [95]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
num_classes = 10
baseMapNum = 32
weight_decay = 1e-4
#model
model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(28,28,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))


In [96]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 28, 28, 32)        896       
_________________________________________________________________
activation_23 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_24 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 32)        0         
__________

In [97]:
#preparing train data

from tensorflow.python.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split

import numpy as np
import cv2
import os
import pandas as pd

a = 28
b = 28

def preprocess(img):
	img=255-np.array(img).astype(np.uint8)
	img = np.array(img).reshape(a,b,3)
	img= img.reshape(1,a*b*3)
	img = img/255
	return img

str=""
i=0
print(os.listdir("../input/data-face/data/DATA"))
final = pd.DataFrame()
for fil in os.listdir("../input/data-face/data/DATA/train"):
            if fil.endswith(".jpg") : 
                str = os.path.join(fil)
                st2 = "../input/data-face/data/DATA/train" +"/" +str
                img = cv2.imread(st2)  
                str =str.split('_')
                str.pop()
                str = [int(i) for i in str]
                img2 = preprocess(img)
                df = pd.DataFrame(img2)
                df[a*b*3]= str[1]*5 + str[2]
                final = final.append(df)
                #print(i)
                i+=1
                if i//1000 ==15 :
                    break
print(i)
final.head()

['val', 'train']
15000


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2313,2314,2315,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352
0,0.686275,0.631373,0.647059,0.682353,0.627451,0.643137,0.725490,0.670588,0.686275,0.772549,0.717647,0.733333,0.694118,0.650980,0.658824,0.529412,0.486275,0.494118,0.407843,0.364706,0.372549,0.372549,0.329412,0.337255,0.231373,0.200000,0.200000,0.180392,0.149020,0.149020,0.125490,0.105882,0.094118,0.113725,0.094118,0.082353,0.117647,0.098039,0.086275,0.117647,...,0.290196,0.294118,0.278431,0.356863,0.360784,0.345098,0.525490,0.529412,0.521569,0.650980,0.650980,0.650980,0.705882,0.705882,0.705882,0.784314,0.784314,0.784314,0.847059,0.847059,0.847059,0.854902,0.850980,0.858824,0.854902,0.850980,0.858824,0.827451,0.823529,0.831373,0.827451,0.823529,0.831373,0.827451,0.823529,0.839216,0.827451,0.823529,0.839216,8
0,0.709804,0.682353,0.529412,0.631373,0.603922,0.450980,0.403922,0.368627,0.219608,0.345098,0.309804,0.160784,0.329412,0.294118,0.145098,0.290196,0.254902,0.105882,0.333333,0.290196,0.141176,0.243137,0.200000,0.050980,0.274510,0.235294,0.078431,0.286275,0.247059,0.090196,0.278431,0.231373,0.074510,0.247059,0.200000,0.043137,0.266667,0.219608,0.062745,0.325490,...,0.521569,0.474510,0.333333,0.584314,0.521569,0.376471,0.572549,0.513725,0.384314,0.352941,0.317647,0.211765,0.188235,0.172549,0.098039,0.184314,0.188235,0.149020,0.231373,0.247059,0.227451,0.450980,0.478431,0.466667,0.788235,0.807843,0.803922,0.780392,0.796078,0.776471,0.835294,0.831373,0.815686,0.878431,0.866667,0.847059,0.901961,0.886275,0.866667,0
0,0.141176,0.164706,0.160784,0.800000,0.807843,0.784314,0.803922,0.776471,0.717647,0.560784,0.494118,0.392157,0.462745,0.349020,0.203922,0.513725,0.360784,0.184314,0.458824,0.274510,0.074510,0.435294,0.239216,0.031373,0.411765,0.223529,0.000000,0.396078,0.215686,0.000000,0.380392,0.211765,0.000000,0.384314,0.235294,0.007843,0.415686,0.278431,0.031373,0.431373,...,0.482353,0.415686,0.247059,0.529412,0.478431,0.305882,0.572549,0.517647,0.352941,0.658824,0.584314,0.435294,0.756863,0.662745,0.521569,0.807843,0.713725,0.572549,0.788235,0.690196,0.564706,0.678431,0.592157,0.482353,0.568627,0.501961,0.400000,0.623529,0.580392,0.501961,0.552941,0.525490,0.466667,0.474510,0.474510,0.427451,0.443137,0.447059,0.411765,3
0,0.913725,0.890196,0.862745,0.862745,0.835294,0.800000,0.858824,0.835294,0.792157,0.933333,0.898039,0.843137,0.980392,0.941176,0.870588,0.992157,0.941176,0.854902,1.000000,0.984314,0.890196,1.000000,1.000000,0.921569,1.000000,1.000000,0.917647,1.000000,1.000000,0.921569,1.000000,1.000000,0.909804,1.000000,1.000000,0.917647,1.000000,1.000000,0.933333,1.000000,...,0.909804,0.847059,0.823529,0.937255,0.905882,0.839216,0.976471,0.952941,0.878431,0.996078,0.984314,0.925490,1.000000,0.980392,0.952941,0.988235,0.976471,0.960784,0.984314,0.964706,0.968627,0.964706,0.964706,0.964706,0.952941,0.956863,0.949020,0.925490,0.941176,0.917647,0.909804,0.929412,0.894118,0.901961,0.917647,0.870588,0.898039,0.913725,0.866667,9
0,0.176471,0.105882,0.109804,0.145098,0.070588,0.050980,0.807843,0.733333,0.674510,1.000000,0.972549,0.866667,0.764706,0.686275,0.525490,0.670588,0.572549,0.376471,0.635294,0.529412,0.305882,0.611765,0.490196,0.250980,0.568627,0.435294,0.200000,0.600000,0.450980,0.223529,0.619608,0.470588,0.243137,0.635294,0.474510,0.258824,0.623529,0.470588,0.254902,0.627451,...,0.760784,0.666667,0.462745,0.760784,0.698039,0.466667,0.788235,0.741176,0.513725,0.858824,0.792157,0.592157,0.909804,0.835294,0.658824,0.933333,0.854902,0.694118,0.925490,0.850980,0.682353,0.909804,0.843137,0.623529,0.882353,0.831373,0.564706,0.933333,0.901961,0.552941,0.941176,0.929412,0.521569,0.870588,0.870588,0.407843,0.619608,0.627451,0.141176,4


In [98]:
#preparing test data

fin = pd.DataFrame()
for fil in os.listdir("../input/data-face/data/DATA/val"):
            if fil.endswith(".jpg") : 
                str = os.path.join(fil)
                st2 = "../input/data-face/data/DATA/val" +"/" +str
                img = cv2.imread(st2)  
                str =str.split('_')
                str.pop()
                str = [int(i) for i in str]
                img2 = preprocess(img)
                df = pd.DataFrame(img2)
                df[a*b*3]= str[1]*5 + str[2]
                fin = fin.append(df)
                #print(i)
                i+=1
fin.head()

24778


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2313,2314,2315,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352
0,0.686275,0.631373,0.647059,0.682353,0.627451,0.643137,0.725490,0.670588,0.686275,0.772549,0.717647,0.733333,0.694118,0.650980,0.658824,0.529412,0.486275,0.494118,0.407843,0.364706,0.372549,0.372549,0.329412,0.337255,0.231373,0.200000,0.200000,0.180392,0.149020,0.149020,0.125490,0.105882,0.094118,0.113725,0.094118,0.082353,0.117647,0.098039,0.086275,0.117647,...,0.290196,0.294118,0.278431,0.356863,0.360784,0.345098,0.525490,0.529412,0.521569,0.650980,0.650980,0.650980,0.705882,0.705882,0.705882,0.784314,0.784314,0.784314,0.847059,0.847059,0.847059,0.854902,0.850980,0.858824,0.854902,0.850980,0.858824,0.827451,0.823529,0.831373,0.827451,0.823529,0.831373,0.827451,0.823529,0.839216,0.827451,0.823529,0.839216,8
0,0.709804,0.682353,0.529412,0.631373,0.603922,0.450980,0.403922,0.368627,0.219608,0.345098,0.309804,0.160784,0.329412,0.294118,0.145098,0.290196,0.254902,0.105882,0.333333,0.290196,0.141176,0.243137,0.200000,0.050980,0.274510,0.235294,0.078431,0.286275,0.247059,0.090196,0.278431,0.231373,0.074510,0.247059,0.200000,0.043137,0.266667,0.219608,0.062745,0.325490,...,0.521569,0.474510,0.333333,0.584314,0.521569,0.376471,0.572549,0.513725,0.384314,0.352941,0.317647,0.211765,0.188235,0.172549,0.098039,0.184314,0.188235,0.149020,0.231373,0.247059,0.227451,0.450980,0.478431,0.466667,0.788235,0.807843,0.803922,0.780392,0.796078,0.776471,0.835294,0.831373,0.815686,0.878431,0.866667,0.847059,0.901961,0.886275,0.866667,0
0,0.913725,0.890196,0.862745,0.862745,0.835294,0.800000,0.858824,0.835294,0.792157,0.933333,0.898039,0.843137,0.980392,0.941176,0.870588,0.992157,0.941176,0.854902,1.000000,0.984314,0.890196,1.000000,1.000000,0.921569,1.000000,1.000000,0.917647,1.000000,1.000000,0.921569,1.000000,1.000000,0.909804,1.000000,1.000000,0.917647,1.000000,1.000000,0.933333,1.000000,...,0.909804,0.847059,0.823529,0.937255,0.905882,0.839216,0.976471,0.952941,0.878431,0.996078,0.984314,0.925490,1.000000,0.980392,0.952941,0.988235,0.976471,0.960784,0.984314,0.964706,0.968627,0.964706,0.964706,0.964706,0.952941,0.956863,0.949020,0.925490,0.941176,0.917647,0.909804,0.929412,0.894118,0.901961,0.917647,0.870588,0.898039,0.913725,0.866667,9
0,0.176471,0.105882,0.109804,0.145098,0.070588,0.050980,0.807843,0.733333,0.674510,1.000000,0.972549,0.866667,0.764706,0.686275,0.525490,0.670588,0.572549,0.376471,0.635294,0.529412,0.305882,0.611765,0.490196,0.250980,0.568627,0.435294,0.200000,0.600000,0.450980,0.223529,0.619608,0.470588,0.243137,0.635294,0.474510,0.258824,0.623529,0.470588,0.254902,0.627451,...,0.760784,0.666667,0.462745,0.760784,0.698039,0.466667,0.788235,0.741176,0.513725,0.858824,0.792157,0.592157,0.909804,0.835294,0.658824,0.933333,0.854902,0.694118,0.925490,0.850980,0.682353,0.909804,0.843137,0.623529,0.882353,0.831373,0.564706,0.933333,0.901961,0.552941,0.941176,0.929412,0.521569,0.870588,0.870588,0.407843,0.619608,0.627451,0.141176,4
0,0.764706,0.831373,0.937255,0.800000,0.831373,0.921569,0.737255,0.725490,0.760784,0.658824,0.584314,0.564706,0.721569,0.592157,0.513725,0.815686,0.650980,0.513725,0.772549,0.600000,0.415686,0.705882,0.533333,0.325490,0.658824,0.525490,0.294118,0.462745,0.349020,0.125490,0.356863,0.278431,0.054902,0.333333,0.262745,0.054902,0.266667,0.203922,0.000000,0.290196,...,1.000000,0.905882,0.796078,1.000000,0.921569,0.827451,1.000000,0.913725,0.835294,1.000000,0.921569,0.839216,1.000000,0.941176,0.874510,0.984314,0.913725,0.847059,0.945098,0.878431,0.827451,0.956863,0.905882,0.850980,0.968627,0.913725,0.866667,1.000000,1.000000,0.960784,0.776471,0.729412,0.690196,0.462745,0.415686,0.376471,0.478431,0.431373,0.392157,6


In [128]:
#Train/ Fit the model on train data
Y = final[2352]
X = final.drop(columns=[2352])
Y = tf.keras.utils.to_categorical(Y,10)
X = np.array(X)
X = X.reshape(15000,28,28,3)
Y_v = fin[2352]
X_v = fin.drop(columns=[2352])
Y_v = tf.keras.utils.to_categorical(Y_v,10)
X_v = np.array(X_v)
X_v = X_v.reshape(9778,28,28,3)
model.fit(X,Y,epochs =50)

Epoch 1/15
15000/15000 [==============================] - 11s 726us/step - loss: 0.3220 - acc: 0.9259
Epoch 2/15
15000/15000 [==============================] - 11s 727us/step - loss: 0.3319 - acc: 0.9234
Epoch 3/15
15000/15000 [==============================] - 11s 720us/step - loss: 0.3282 - acc: 0.9223
Epoch 4/15
15000/15000 [==============================] - 11s 726us/step - loss: 0.3261 - acc: 0.9235
Epoch 5/15
15000/15000 [==============================] - 11s 725us/step - loss: 0.3249 - acc: 0.9248
Epoch 6/15
15000/15000 [==============================] - 11s 725us/step - loss: 0.3238 - acc: 0.9243
Epoch 7/15
15000/15000 [==============================] - 11s 724us/step - loss: 0.3239 - acc: 0.9267
Epoch 8/15
15000/15000 [==============================] - 11s 734us/step - loss: 0.3234 - acc: 0.9270
Epoch 9/15
15000/15000 [==============================] - 11s 727us/step - loss: 0.3306 - acc: 0.9235
Epoch 10/15
15000/15000 [==============================] - 11s 727us/step - loss: 

In [132]:
#Evaluating on test data
model.save('weights.h5')
pred= model.evaluate( X_v[:7150],Y_v[:7150])
print("On test data:")
print('Loss: ',pred[0])
acc = pred[1]*100
print('Accuracy: ',acc,"%")

7150/7150 [==============================] - 2s 300us/step
On test data:
Loss:  0.384139076346284
Accuracy:  94.19580419413694 %


In [133]:
#Notes
print('Has scope for Improvements')

Has scope for Improvements
